In [ ]:
import cv2
import glob
import os
import numpy as np;
import pandas as pd;

path='D:/Evaporation_paper/Wenzel_Boiling_Cassie_transition/Video2framespy'
Volume=[]
Time=[]
#images=[cv2.imread(file) for file in glob.glob('F:/Droplet_vibration/Self_propulsion Droplet on thin hot oil film/Video2Frames/*.jpg')]
for imagepath in glob.glob(path+'/120Cp_1_4/*.jpg'):
    #obtain the path of the picture
    
    #with the picture path, transfer the picture into grayscale matrix
    im_gray=cv2.imread(str(imagepath),cv2.IMREAD_GRAYSCALE)
    
    #set threshold
    thresh=100
    
    #transfer grayscale into binaryscale
    th, im_b = cv2.threshold(im_gray, thresh, 255, cv2.THRESH_BINARY_INV)
    
    # Copy the thresholded image.
    im_floodfill = im_b.copy()
    
    # Mask used to flood filling.
    # Notice the size needs to be 2 pixels than the image.
    h, w = im_gray.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    
    # Floodfill from point (0, 0)
    cv2.floodFill(im_floodfill, mask, (20,20), 255);
    
    # Invert floodfilled image
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)
    
    # Combine the two images to get the foreground.
    # Get the images with holes filled
    im_out = im_b | im_floodfill_inv
    
    
    im = im_out.copy()/255
    file=pd.DataFrame(im*10)
    file.to_csv(path+'/a.csv')
    
    
    
    # Calculate the width of the droplet 
    width=sum(im.T)[:]
    r=int(width.shape[0])
    v=0
    # Calsulate the volume of droplet in the chosen image 
    for i in range(30,400):
        v=v+width[i-1]**2*3.1415926/4*((0.312/35)**3)
    Volume=np.append(Volume,v)
    Time=np.append(Time,int(os.path.basename(imagepath).split('.')[0]))  
    
    #combine the Time and Volume
    point=zip(Time,Volume)    #combine
    cc=sorted(point)          #sort the list with ascending Time
    f=pd.DataFrame(cc)
    f.to_csv(path+'/Output.csv',index=False, header=False)
    
    #cv2.imshow(os.path.basename(imagepath), im)
    #print(os.path.basename(imagepath).split('.')[0])
    
cv2.waitKey(0)